In [37]:
import pandas as pd
import numpy as np

In [38]:
training=pd.read_csv('house-prices-advanced-regression-techniques/train.csv',index_col='Id')
training=training.replace('NA ',np.nan)
d=training.shape
temp=training.isna().sum()[training.isna().sum()>=(d[0]*0.75)].index
print(temp)
na_l=training.columns[training.isna().any()].tolist() # columns with nan in them
me=training[na_l].select_dtypes("float64").median()
for item in temp:
    m=training[item].isnull().values
    training.insert((d[1]-1),("Is there a "+item+" ?"),~m)
    training[("Is there a "+item+" ?")]=training[("Is there a "+item+" ?")].astype('int64')
for item in training[na_l].select_dtypes("object"):
    if item not in temp:
        tem=training[item].value_counts()
        tem1=tem.idxmax()
        training[item]=tem1
for item in me.index:
    training[item]=training[item].replace(np.nan,me[item])
training=training.drop(columns=temp)
col=training.pop('SalePrice')
d=training.shape
training.insert(d[1],'SalePrice',col)
print(temp)
training.head()

Index(['Alley', 'PoolQC', 'Fence', 'MiscFeature'], dtype='object')
Index(['Alley', 'PoolQC', 'Fence', 'MiscFeature'], dtype='object')


,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,MiscVal,MoSold,YrSold,SaleType,SaleCondition,Is there a MiscFeature ?,Is there a Fence ?,Is there a PoolQC ?,Is there a Alley ?,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,2,2008,WD,Normal,0,0,0,0,208500
2,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,...,0,5,2007,WD,Normal,0,0,0,0,181500
3,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,0,9,2008,WD,Normal,0,0,0,0,223500
4,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,...,0,2,2006,WD,Abnorml,0,0,0,0,140000
5,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,...,0,12,2008,WD,Normal,0,0,0,0,250000


In [39]:
test_x=pd.read_csv('house-prices-advanced-regression-techniques/test.csv',index_col='Id')
test_y=pd.read_csv('house-prices-advanced-regression-techniques/sample_submission.csv',index_col='Id')
test_x=test_x.replace('NA ',np.nan)
d=test_x.shape
temp=test_x.isna().sum()[test_x.isna().sum()>=(d[0]*0.75)].index
print(temp)
na_l=test_x.columns[test_x.isna().any()].tolist() # columns with nan in them
me=test_x[na_l].select_dtypes("float64").median()
for item in temp:
    m=test_x[item].isnull().values
    test_x.insert((d[1]-1),("Is there a "+item+" ?"),~m)
    test_x[("Is there a "+item+" ?")]=test_x[("Is there a "+item+" ?")].astype('int64')
for item in test_x[na_l].select_dtypes("object"):
    if item not in temp:
        tem=test_x[item].value_counts()
        tem1=tem.idxmax()
        test_x[item]=tem1
for item in me.index:
    test_x[item]=test_x[item].replace(np.nan,me[item])
test_x=test_x.drop(columns=temp)
print(temp)
test_x.head()

Index(['Alley', 'PoolQC', 'Fence', 'MiscFeature'], dtype='object')
Index(['Alley', 'PoolQC', 'Fence', 'MiscFeature'], dtype='object')


,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,PoolArea,MiscVal,MoSold,YrSold,SaleType,Is there a MiscFeature ?,Is there a Fence ?,Is there a PoolQC ?,Is there a Alley ?,SaleCondition
Id,,,,,,,,,,,,,,,,,,,,,
1461,20,RL,80.0,11622,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,6,2010,WD,0,1,0,0,Normal
1462,20,RL,81.0,14267,Pave,IR1,Lvl,AllPub,Corner,Gtl,...,0,12500,6,2010,WD,1,0,0,0,Normal
1463,60,RL,74.0,13830,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,0,0,3,2010,WD,0,1,0,0,Normal
1464,60,RL,78.0,9978,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,0,0,6,2010,WD,0,0,0,0,Normal
1465,120,RL,43.0,5005,Pave,IR1,HLS,AllPub,Inside,Gtl,...,0,0,1,2010,WD,0,0,0,0,Normal


In [40]:
train_x=training.iloc[:,:-1]
train_y=training.iloc[:,-1]